In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import date
# import numpy as np

# df = pd.DataFrame({"id": ['1', '2', '3', '4', '5', '6', '7', '8'],

#                     "prepro": ['prepresp1', 'prepre1', 'prepresub1', 'prepresp2', 'prepre2', 'prepresub2', 'prepro1', 'prepro2'],

#                     "insert": [np.nan, np.nan, 'insert', np.nan, 'insert', np.nan, np.nan,  'insertcool'],

#                    "pre": ['presp1', 'presp1', 'presub1', 'presp2', 'presp2', 'presub2', np.nan, np.nan],

#                    "sp": ['sp1', np.nan, 'spsub1', 'sp2', np.nan, 'spsub2', np.nan, np.nan] ,

#                    "sub": [np.nan, np.nan, 'sub1', np.nan, np.nan, 'sub2', np.nan, np.nan]})

# df

In [2]:
observations_path = 'data/observations-285432.csv'
# start_date = date(2021,10,2)
start_date = 'min'
# finish_date = date(2019,5,30)
finish_date = 'max'

In [3]:
def prepare_df(observations_path, start_date, finish_date):
    
    df_full = pd.read_csv(observations_path)
    df = df_full.loc[:,'taxon_id':'taxon_form_name'].copy()
    df.insert(0,'created_at','')
    df['created_at'] = pd.to_datetime(df_full['created_at']).dt.date
    del(df_full)
    start_date = min(df['created_at']) if start_date == 'min' else start_date
    finish_date = max(df['created_at']) if finish_date == 'max' else finish_date
    return df, start_date, finish_date

In [4]:
df_taxons, start_date, finish_date = prepare_df(observations_path, start_date, finish_date)

In [5]:
df_taxons

,created_at,taxon_id,taxon_kingdom_name,taxon_phylum_name,taxon_subphylum_name,taxon_superclass_name,taxon_class_name,taxon_subclass_name,taxon_superorder_name,taxon_order_name,taxon_suborder_name,taxon_superfamily_name,taxon_family_name,taxon_subfamily_name,taxon_supertribe_name,taxon_tribe_name,taxon_subtribe_name,taxon_genus_name,taxon_genushybrid_name,taxon_species_name,taxon_hybrid_name,taxon_subspecies_name,taxon_variety_name,taxon_form_name
0,2018-09-24,1062676,Fungi,Ascomycota,Pezizomycotina,NaN,Pezizomycetes,NaN,NaN,Pezizales,NaN,NaN,Morchellaceae,NaN,NaN,NaN,NaN,Morchella,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-09-24,1070919,Animalia,Arthropoda,Chelicerata,NaN,Arachnida,NaN,NaN,Araneae,Araneomorphae,Salticoidea,Salticidae,Salticinae,NaN,Sitticini,NaN,Attulus,NaN,Attulus floricola,NaN,NaN,NaN,NaN
2,2019-05-26,373470,Animalia,Arthropoda,Hexapoda,NaN,Insecta,Pterygota,NaN,Coleoptera,Polyphaga,Cleroidea,Melyridae,Malachiinae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-05-28,966205,Plantae,Tracheophyta,Angiospermae,NaN,Magnoliopsida,NaN,NaN,Fagales,NaN,NaN,Betulaceae,NaN,NaN,NaN,NaN,Alnus,NaN,Alnus glutinosa,NaN,NaN,NaN,NaN
4,2019-05-28,51610,Plantae,Tracheophyta,Angiospermae,NaN,Magnoliopsida,NaN,NaN,Lamiales,NaN,NaN,Plantaginaceae,NaN,NaN,Veroniceae,NaN,Veronica,NaN,Veronica chamaedrys,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,2022-12-15,40151,Animalia,Chordata,Vertebrata,NaN,Mammalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
931,2022-12-15,40151,Animalia,Chordata,Vertebrata,NaN,Mammalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932,2022-12-15,40151,Animalia,Chordata,Vertebrata,NaN,Mammalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,2022-12-15,40151,Animalia,Chordata,Vertebrata,NaN,Mammalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def get_taxons_dict_to_date(df, date_to):

    df = df_taxons[df_taxons['created_at'] <= date_to].copy()
    df.drop('created_at', axis=1, inplace=True)
    
    taxons_full = df.columns
    df.dropna(axis=1, how='all', inplace=True)
    not_empty_taxons = df.columns

    last_levels = df.apply(lambda x: x.last_valid_index(), axis=1)
    levels = df.columns[df.columns.isin(last_levels.unique())].to_list()
    df = df[['taxon_id']+levels].copy()

    empty_taxons = set(taxons_full) - set(not_empty_taxons)
    full_taxons = set(not_empty_taxons) - set(df.columns)

    col_qty = len(df.columns)
    last_col = df.columns[col_qty-1]

    taxons_dict = dict()

    for i in range(1, col_qty):
        level_name = df.columns[i]
        taxons_dict[level_name] = dict()

    for i in range(1, col_qty):
        level_name = df.columns[i]
        level_taxons = df[last_levels == level_name][level_name].to_list()
        level_ids = df[last_levels == level_name].iloc[:,0].to_list()
        taxons_dict[level_name].update(zip(level_ids, level_taxons))
        i += 1

    return taxons_dict, df, empty_taxons, full_taxons

In [7]:
taxons_dict_start, df_taxons_clear_start, empty_taxons_start, full_taxons_start = get_taxons_dict_to_date(df_taxons, start_date)
taxons_dict_finish, df_taxons_clear_finish, empty_taxons_finish, full_taxons_finish = get_taxons_dict_to_date(df_taxons, finish_date)

In [ ]:
for level in taxons_dict.keys():
    print('level:', level, ', q-ty: ', len(taxons_dict[level]))

level: taxon_genus_name , q-ty:  1
level: taxon_species_name , q-ty:  1
